In [74]:
import sys
import os

sys.path.insert(0, os.path.dirname(os.getcwd()))

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, rand, split, explode, regexp_replace, corr, array_contains, contains, round

from schemas.dataframes import get_episode_df, get_basics_df, get_akas_df, get_crew_df, get_principals_df, get_ratings_df, get_name_df

In [2]:
spark_session = (SparkSession.builder
                             .master('local')
                             .appName('test app')
                             .config(conf=SparkConf())
                             .getOrCreate())

In [3]:
title_episode = get_episode_df(spark_session)

title_basic = get_basics_df(spark_session)

title_akas = get_akas_df(spark_session)

rating = get_ratings_df(spark_session)

Query 1 (10 випадкових серіалів, які мають рейтинг більше 8 за всі сезони та більше 50 серій, але менше 250)

In [75]:
merged_data = (
    title_episode
    .join(title_basic, title_episode['parentTconst'] == title_basic['tconst'], 'inner')
    .join(rating, title_basic['tconst'] == rating['tconst'], 'inner')
)

grouped_data = (
    merged_data
    .groupBy("primaryTitle")
    .agg({"seasonNumber": "max", "episodeNumber": "count", "averageRating": "avg"})
    .withColumnRenamed("max(seasonNumber)", "MaxSeason")
    .withColumnRenamed("count(episodeNumber)", "TotalEpisodes")
    .withColumnRenamed("avg(averageRating)", "AverageRating")
    .withColumn("AverageRating", round(col("AverageRating"), 1))
)

filtered_data = (
    grouped_data
    .filter((col("AverageRating") > 8) & (col("TotalEpisodes") > 50) & (col("TotalEpisodes") < 250) & (col("MaxSeason") > 5))
    .orderBy(rand())
    .limit(10)
)

filtered_data.show()

+--------------------+-------------+---------+-------------+
|        primaryTitle|AverageRating|MaxSeason|TotalEpisodes|
+--------------------+-------------+---------+-------------+
|          The Xcorps|          8.9|        6|           64|
|   Who Dunit Theater|          8.1|        6|           64|
|         Fantorangen|          9.3|       12|          154|
|            Amrutham|          9.6|        6|           61|
|          The League|          8.3|        7|           84|
|    The Taste Brasil|          8.8|        6|           60|
|      Blameitonjorge|          8.1|        8|          104|
|       Who is J.O.B?|          9.2|        7|           73|
|Feuer & Flamme: M...|          9.0|        7|           51|
|           The Crown|          8.6|        6|           60|
+--------------------+-------------+---------+-------------+



Query 2 (найбільш тривалі серіали\фільми за жанрами від 2020 до 2023 року)

In [87]:
start_date = "2020"
end_date = "2023"

filtered_movies = (
    title_basic.filter((col("startYear") >= start_date) & (col("startYear") < end_date) & (col("genres").contains("\\N")))
               .select("tconst", explode("genres").alias("genre"), "runtimeMinutes")
               .groupBy("genre")
               .agg({"runtimeMinutes": "avg"}).limit(10)
               .orderBy(col("avg(runtimeMinutes)").desc())
)

filtered_movies.show()

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "contains(genres, \N)" due to data type mismatch: Parameter 1 requires the "STRING" type, however "genres" has the type "ARRAY<STRING>".;
'Filter (((startYear#13 >= cast(2020 as int)) AND (startYear#13 < cast(2023 as int))) AND Contains(genres#26, \N))
+- Project [tconst#8, titleType#9, primaryTitle#10, originalTitle#11, isAdult#12, startYear#13, endYear#14, runtimeMinutes#15, split(genres#16, ,, -1) AS genres#26]
   +- Relation [tconst#8,titleType#9,primaryTitle#10,originalTitle#11,isAdult#12,startYear#13,endYear#14,runtimeMinutes#15,genres#16] csv


In [10]:
start_date = "2020"
end_date = "2023"

filtered_movies = (
    title_basic.filter((col("startYear") >= start_date) & (col("startYear") < end_date))
               .select("tconst", explode("genres").alias("genre"), "runtimeMinutes")
               .groupBy("genre")
               .agg({"runtimeMinutes": "avg"}).limit(10)
               .orderBy(col("avg(runtimeMinutes)").desc())
)

filtered_movies.show()

+-----------+-------------------+
|      genre|avg(runtimeMinutes)|
+-----------+-------------------+
|        War| 57.662944162436546|
| Reality-TV|  54.77462393026099|
|   Thriller| 51.212988442487614|
|      Crime|  49.19968429360694|
|    Romance|  47.95665236051502|
|Documentary| 44.673557804663204|
|  Adventure| 44.585649044043656|
|         \N|  43.35081764887381|
|      Drama| 43.013921909560814|
|     Family|  34.47700718190588|
+-----------+-------------------+



Query 3 - Фільми, які містять слово "dog" в оригінальному заголовку

In [79]:
dog_movies = (
    title_akas
    .join(title_basic, title_akas['titleId'] == title_basic['tconst'], 'inner')
    .filter(  (col("titleType") == "movie") 
            & (col("originalTitle").contains("Penguins of Madagascar")) 
            & (col("startYear") > 1950))
    .orderBy(rand())
    .select("primaryTitle", "originalTitle", "language", "startYear", "genres")
    .limit(10)
)

dog_movies.show()

+--------------------+--------------------+--------+---------+--------------------+
|        primaryTitle|       originalTitle|language|startYear|              genres|
+--------------------+--------------------+--------+---------+--------------------+
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      en|     2014|[Action, Adventur...|
|Penguins of Madag...|Penguins of Madag...|      \N|     2014|[Action, Adven

In [71]:
dog_movies = (
    title_akas
    .join(title_basic, title_akas['titleId'] == title_basic['tconst'], 'inner')
    .filter(  (col("titleType") == "movie") 
            & (col("originalTitle").contains("dog")) 
            & (col("startYear") > 1950))
    .orderBy(rand())
    .select("primaryTitle", "originalTitle", "language", "startYear", "genres")
    .limit(10)
)

dog_movies.show()

+--------------------+--------------------+--------+---------+--------------------+
|        primaryTitle|       originalTitle|language|startYear|              genres|
+--------------------+--------------------+--------+---------+--------------------+
|A Bulldog for Chr...|A Bulldog for Chr...|      \N|     2013|[Comedy, Drama, F...|
|  Viva the Underdogs|  Viva the Underdogs|      \N|     2020|       [Documentary]|
|            Upperdog|            Upperdog|      \N|     2009|     [Comedy, Drama]|
|           Hell Dogs|        Heru doggusu|      en|     2022|[Action, Crime, D...|
|       Does It Hurt?|Boli li? Prvata b...|      \N|     2007|       [Documentary]|
|     Happy Birthday!|Boldog születésna...|      en|     2003|   [Comedy, Romance]|
|        Baby Bulldog|        Baby Bulldog|      \N|     2020|            [Family]|
|            Underdog|            Underdog|      \N|     2007|[Action, Adventur...|
|The Dancin' Bulldogs|The Dancin' Bulldogs|      \N|     2020|       [Docume

Query 4 Найпопулярніші фільми продубльовані іспанською мовою після 2000 року

In [33]:
espaniol_movies = (
    title_akas
    .join(title_basic, title_akas['titleId'] == title_basic['tconst'])
    .join(rating, title_basic['tconst'] == rating['tconst'])
    .filter((col("language") == "es") & (col("startYear") > 2000) & (col("titleType") == "movie"))
    .select("primaryTitle", "originalTitle", "language", "startYear", "genres", "averageRating")
    .orderBy(col("averageRating").desc())
    .limit(10)
)

espaniol_movies.show()

+--------------------+--------------------+--------+---------+--------------------+-------------+
|        primaryTitle|       originalTitle|language|startYear|              genres|averageRating|
+--------------------+--------------------+--------+---------+--------------------+-------------+
|  Heavier Is the Sky| Mais Pesado é o Céu|      es|     2023|             [Drama]|         10.0|
|  Carving the Divine|  Carving the Divine|      es|     2019|       [Documentary]|          9.9|
|Catholicism: The ...|Catholicism: The ...|      es|     2016|       [Documentary]|          9.4|
|   Mein fremdes Land|   Mein fremdes Land|      es|     2021|       [Documentary]|          9.3|
|  Strangers to Peace|  Strangers to Peace|      es|     2022|       [Documentary]|          9.0|
|              Cringe|              Cringe|      es|     2020|[Comedy, Document...|          8.8|
|           Canceling|           Canceling|      es|     2020|            [Comedy]|          8.8|
|    Under Rich Eart

Query 5 кореляція між тривалістю часу фільму та рейтингу

In [34]:
correlation_result = (rating
                      .join(title_basic, rating['tconst'] == title_basic['tconst'], 'inner')
                      .select("runtimeMinutes", "averageRating")
                      .na.drop()
                      .filter((col("titleType") == "movie"))
                      .select(corr("runtimeMinutes", "averageRating").alias("correlation between runtime and rating")))

correlation_result.show()

+--------------------------------------+
|correlation between runtime and rating|
+--------------------------------------+
|                  0.008873044893575995|
+--------------------------------------+



Query 6 кількість дубльованих фільмів німецькою мовою від 2010 року

In [32]:
german_movies = (
    title_akas
    .filter(col("language") == "de")
    .join(title_basic, title_akas['titleId'] == title_basic['tconst'], 'inner')
    .filter(  (col("startYear") >= 2010) 
            & (col("runtimeMinutes") > 20)
            & (col("titleType") == "movie"))
)

german_movies_by_year = (
    german_movies
    .groupBy("startYear")
    .count()
    .withColumnRenamed("count", "amount_of_films")
    .orderBy("startYear")
)

german_movies_by_year.show()

+---------+---------------+
|startYear|amount_of_films|
+---------+---------------+
|     2010|             11|
|     2011|             13|
|     2012|             18|
|     2013|             26|
|     2014|             22|
|     2015|             34|
|     2016|             17|
|     2017|             25|
|     2018|             32|
|     2019|             32|
|     2020|             37|
|     2021|             82|
|     2022|             78|
|     2023|             58|
|     2024|              1|
+---------+---------------+

